In [ ]:
# Full Image Processing Pipeline - Step by Step

Complete medical image processing pipeline with all stages:

1. **Load Images** (Projection, Gain, Dark)
2. **Flat Field Correction (FFC)** - Remove detector non-uniformity
3. **Spatial Calibration** - Correct lens distortion & crop ROI
4. **BEMD Decomposition** - Extract frequency components (BIMFs)
5. **Homomorphic Filtering** - Normalize brightness & enhance contrast
6. **Nonlinear Filtering** - Denoise & reconstruct from BIMFs
7. **Gamma Correction** - Adjust brightness
8. **CLAHE Enhancement** - Adaptive local contrast enhancement
9. **Image Quality Metrics** - CII, Entropy, EME evaluation
10. **Normalize & Resize** - 16-bit output at target resolution
11. **Final Comparison** - All stages side-by-side

In [ ]:
# ============================================================
# Step 0: Imports & Configuration
# ============================================================
import os, gc, sys
import cv2
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt

# Add project root to path
sys.path.insert(0, os.path.abspath("."))

from image_pipeline import (
    PipelineConfig,
    FlatFieldCorrection,
    SpatialCalibration,
    BEMD,
    HomomorphicFilter,
    NonlinearFilter,
    ImageEnhancer,
    ImageMetrics,
    ImageResizer,
)

# ---------- Image Paths ----------
PROJ_IMG_PATH  = ""  # <-- SET your raw projection image path (.mdn / .tiff)
GAIN_IMG_PATH  = "datacitra/Gain/Bed/80_50_0,50.mdn"
DARK_IMG_PATH  = "datacitra/Dark/Bed/dark.mdn"
CALIB_PATH     = "datacitra/Kalibrasi/bed_44_35.npz"

# ---------- FFC ----------
FFC_MEDIAN_SIZE = 7

# ---------- BEMD ----------
BEMD_MAX_ITER     = 1
BEMD_THRESHOLD    = 1.0
BEMD_WINDOW_SIZE  = 32
BEMD_EXTREMA_COUNT = 10

# ---------- Homomorphic filter ----------
D0 = 30
RH = 2.0
RL = 0.5

# ---------- Enhancement ----------
GAMMA      = 0.8
CLAHE_CLIP = 3.0
CLAHE_TILE = (8, 8)

# ---------- Nonlinear filter ----------
DENOISE_R    = 1
DENOISE_BETA = 0.5

# ---------- Output ----------
OUTPUT_WIDTH = 4096

print("Step 0: Imports & Configuration loaded")
print(f"   Projection : {PROJ_IMG_PATH}")
print(f"   Gain       : {GAIN_IMG_PATH}")
print(f"   Dark       : {DARK_IMG_PATH}")
print(f"   Calibration: {CALIB_PATH}")
print(f"   FFC median : {FFC_MEDIAN_SIZE}")
print(f"   BEMD       : max_iter={BEMD_MAX_ITER}, threshold={BEMD_THRESHOLD}, window={BEMD_WINDOW_SIZE}, extrema={BEMD_EXTREMA_COUNT}")
print(f"   Homomorphic: D0={D0}, Rh={RH}, Rl={RL}")
print(f"   Enhancement: gamma={GAMMA}, CLAHE clip={CLAHE_CLIP}, tile={CLAHE_TILE}")
print(f"   Nonlinear  : r={DENOISE_R}, beta={DENOISE_BETA}")
print(f"   Output width: {OUTPUT_WIDTH}")

## Step 1: Load Images

Load the three images required for Flat Field Correction:
- **Projection image** - the raw X-ray capture
- **Gain image** - detector response to uniform illumination
- **Dark image** - detector noise with no illumination

In [ ]:
# ============================================================
# Step 1: Load Images (Projection, Gain, Dark)
# ============================================================
proj_image = cv2.imread(PROJ_IMG_PATH, cv2.IMREAD_UNCHANGED)
gain_image = cv2.imread(GAIN_IMG_PATH, cv2.IMREAD_UNCHANGED)
dark_image = cv2.imread(DARK_IMG_PATH, cv2.IMREAD_UNCHANGED)

for name, img, path in [("Projection", proj_image, PROJ_IMG_PATH),
                         ("Gain", gain_image, GAIN_IMG_PATH),
                         ("Dark", dark_image, DARK_IMG_PATH)]:
    if img is None:
        raise FileNotFoundError(f"Could not load {name} image: {path}")

print("Step 1: All images loaded successfully")
for name, img in [("Projection", proj_image), ("Gain", gain_image), ("Dark", dark_image)]:
    print(f"   {name:12s}: shape={img.shape}, dtype={img.dtype}, "
          f"min={img.min()}, max={img.max()}, mean={img.mean():.2f}")

# Display all three images
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for ax, (title, img) in zip(axes, [("Projection", proj_image),
                                     ("Gain", gain_image),
                                     ("Dark", dark_image)]):
    ax.imshow(img, cmap='gray')
    ax.set_title(f'{title} ({img.shape[1]}x{img.shape[0]})')
    ax.axis('off')

plt.suptitle('Step 1: Loaded Images', fontsize=14)
plt.tight_layout()
plt.show()

# Histograms
fig, axes = plt.subplots(1, 3, figsize=(18, 3))
for ax, (title, img, color) in zip(axes, [("Projection", proj_image, 'steelblue'),
                                           ("Gain", gain_image, 'coral'),
                                           ("Dark", dark_image, 'seagreen')]):
    ax.hist(img.ravel(), bins=256, color=color, alpha=0.7)
    ax.set_title(f'{title} Histogram')
    ax.set_xlabel('Pixel Intensity')

plt.suptitle('Step 1: Image Histograms', fontsize=14)
plt.tight_layout()
plt.show()

## Step 2: Flat Field Correction (FFC)

FFC removes detector non-uniformity using the formula:

$$\mu = -\ln\!\left(\frac{I_{proj} - I_{dark}}{I_{gain} - I_{dark}}\right)$$

Images are normalized to [0,1], median-filtered, then the correction is computed on GPU.

In [ ]:
# ============================================================
# Step 2: Flat Field Correction (FFC)
# ============================================================
ffc = FlatFieldCorrection(median_filter_size=FFC_MEDIAN_SIZE)
ffc_image_gpu = ffc.apply(proj_image, gain_image, dark_image)

# Bring back to CPU for display
ffc_image = ffc_image_gpu.get()

print("Step 2: Flat Field Correction completed")
print(f"   Median filter size: {FFC_MEDIAN_SIZE}")
print(f"   Output shape: {ffc_image.shape}")
print(f"   Output dtype: {ffc_image.dtype}")
print(f"   Min: {ffc_image.min():.6f}, Max: {ffc_image.max():.6f}, Mean: {ffc_image.mean():.6f}")

# Display projection vs FFC result
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(proj_image, cmap='gray')
axes[0].set_title('Raw Projection')
axes[0].axis('off')

axes[1].imshow(ffc_image, cmap='gray')
axes[1].set_title('After Flat Field Correction')
axes[1].axis('off')

plt.suptitle('Step 2: Flat Field Correction', fontsize=14)
plt.tight_layout()
plt.show()

# Histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 3))
axes[0].hist(proj_image.ravel(), bins=256, color='steelblue', alpha=0.7)
axes[0].set_title('Raw Projection Histogram')
axes[0].set_xlabel('Pixel Intensity')

axes[1].hist(ffc_image.ravel(), bins=256, color='coral', alpha=0.7)
axes[1].set_title('FFC Result Histogram')
axes[1].set_xlabel('Pixel Intensity')

plt.suptitle('Step 2: Histogram Before/After FFC', fontsize=14)
plt.tight_layout()
plt.show()

## Step 3: Spatial Calibration

Corrects lens distortion using camera matrix and distortion coefficients from a pre-computed calibration file (.npz). The image is undistorted and cropped to the valid region of interest (ROI).

In [ ]:
# ============================================================
# Step 3: Spatial Calibration
# ============================================================
calibrator = SpatialCalibration(CALIB_PATH)
calibrated_image = calibrator.apply(ffc_image_gpu)

print("Step 3: Spatial Calibration completed")
print(f"   Calibration file: {CALIB_PATH}")
print(f"   Before: {ffc_image.shape}")
print(f"   After : {calibrated_image.shape}")
print(f"   Dtype : {calibrated_image.dtype}")
print(f"   Min: {calibrated_image.min():.6f}, Max: {calibrated_image.max():.6f}, Mean: {calibrated_image.mean():.6f}")
print(f"   ROI   : {calibrator.roi}")

# Display FFC vs Calibrated
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(ffc_image, cmap='gray')
axes[0].set_title(f'FFC Result ({ffc_image.shape[1]}x{ffc_image.shape[0]})')
axes[0].axis('off')

axes[1].imshow(calibrated_image, cmap='gray')
axes[1].set_title(f'Calibrated & Cropped ({calibrated_image.shape[1]}x{calibrated_image.shape[0]})')
axes[1].axis('off')

plt.suptitle('Step 3: Spatial Calibration', fontsize=14)
plt.tight_layout()
plt.show()

## Step 4: BEMD Decomposition

Bidimensional Empirical Mode Decomposition extracts **BIMFs** (Bidimensional Intrinsic Mode Functions) from the calibrated image. Each BIMF captures different frequency components. The energies of each BIMF are calculated for later use in nonlinear filtering.

In [ ]:
# ============================================================
# Step 4: BEMD Decomposition
# ============================================================
bemd = BEMD(
    max_iterations=BEMD_MAX_ITER,
    threshold=BEMD_THRESHOLD,
    initial_window_size=BEMD_WINDOW_SIZE,
    local_extrema_count=BEMD_EXTREMA_COUNT,
)

# Convert calibrated image to CuPy for GPU processing
image_gpu = cp.asarray(calibrated_image.astype(np.float64))

print(f"Step 4: Running BEMD decomposition...")
print(f"   Input shape: {image_gpu.shape}, dtype: {image_gpu.dtype}")

bimfs = bemd.decompose(image_gpu)
energies = BEMD.calculate_energies(bimfs)

print(f"\nStep 4: BEMD completed")
print(f"   Number of BIMFs extracted: {len(bimfs)}")
for i, (bimf, energy) in enumerate(zip(bimfs, energies)):
    bimf_np = bimf.get()
    print(f"   BIMF {i}: shape={bimf_np.shape}, energy={energy:.4f}, "
          f"min={bimf_np.min():.4f}, max={bimf_np.max():.4f}")

# Display BIMFs
n_bimfs = len(bimfs)
cols = min(n_bimfs, 4)
rows = (n_bimfs + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
if n_bimfs == 1:
    axes = np.array([axes])
axes = np.atleast_2d(axes)

for i, bimf in enumerate(bimfs):
    r, c = divmod(i, cols)
    ax = axes[r, c]
    ax.imshow(bimf.get(), cmap='gray')
    ax.set_title(f'BIMF {i} (E={energies[i]:.2f})')
    ax.axis('off')

# Hide unused subplots
for i in range(n_bimfs, rows * cols):
    r, c = divmod(i, cols)
    axes[r, c].axis('off')

plt.suptitle('Step 4: Extracted BIMFs', fontsize=14)
plt.tight_layout()
plt.show()

# Energy bar chart
fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.bar(range(n_bimfs), energies, color='coral')
ax.set_xlabel('BIMF Index')
ax.set_ylabel('Energy')
ax.set_title('Step 4: BIMF Energies')
ax.set_xticks(range(n_bimfs))
plt.tight_layout()
plt.show()

## Step 5: Homomorphic Filtering

Homomorphic filtering operates in the frequency domain to simultaneously normalize brightness and increase contrast. It applies a logarithmic transform, performs Fourier-domain high-frequency emphasis, then reverses with an exponential transform.

In [ ]:
# ============================================================
# Step 5: Homomorphic Filtering
# ============================================================
hf = HomomorphicFilter(d0=D0, rh=RH, rl=RL)
filtered_image = hf.apply(calibrated_image)

print(f"Step 5: Homomorphic Filter applied")
print(f"   D0={D0}, Rh={RH}, Rl={RL}")
print(f"   Output shape: {filtered_image.shape}")
print(f"   Output dtype: {filtered_image.dtype}")
print(f"   Min: {filtered_image.min()}, Max: {filtered_image.max()}, Mean: {filtered_image.mean():.2f}")

# Display calibrated vs filtered
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(calibrated_image, cmap='gray')
axes[0].set_title('Calibrated Image')
axes[0].axis('off')

axes[1].imshow(filtered_image, cmap='gray')
axes[1].set_title(f'Homomorphic Filtered (D0={D0}, Rh={RH}, Rl={RL})')
axes[1].axis('off')

plt.suptitle('Step 5: Homomorphic Filtering', fontsize=14)
plt.tight_layout()
plt.show()

# Histogram comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 3))
axes[0].hist(calibrated_image.ravel(), bins=256, color='steelblue', alpha=0.7)
axes[0].set_title('Calibrated Histogram')
axes[0].set_xlabel('Pixel Intensity')

axes[1].hist(filtered_image.ravel(), bins=256, color='coral', alpha=0.7)
axes[1].set_title('Filtered Histogram')
axes[1].set_xlabel('Pixel Intensity')

plt.suptitle('Step 5: Histogram Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## Step 6: Nonlinear Filtering (Denoise & Reconstruct)

The nonlinear filter sorts BIMFs by energy, applies bilateral filtering to the **R lowest-energy** BIMFs (noise components), then reconstructs the image by combining denoised + original BIMFs + weighted homomorphic-filtered residual.

In [ ]:
# ============================================================
# Step 6: Nonlinear Filtering (Denoise & Reconstruct)
# ============================================================
nlf = NonlinearFilter(r=DENOISE_R, beta=DENOISE_BETA)
denoised_image = nlf.denoise(bimfs, energies, filtered_image)

print(f"Step 6: Nonlinear Filtering completed")
print(f"   r={DENOISE_R} (lowest-energy BIMFs denoised), beta={DENOISE_BETA}")
print(f"   Sorted energy indices: {np.argsort(energies).tolist()}")
print(f"   Output shape: {denoised_image.shape}")
print(f"   Output dtype: {denoised_image.dtype}")
print(f"   Min: {denoised_image.min():.4f}, Max: {denoised_image.max():.4f}, Mean: {denoised_image.mean():.4f}")

# Display
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(calibrated_image, cmap='gray')
axes[0].set_title('Calibrated')
axes[0].axis('off')

axes[1].imshow(filtered_image, cmap='gray')
axes[1].set_title('Homomorphic Filtered')
axes[1].axis('off')

axes[2].imshow(denoised_image, cmap='gray')
axes[2].set_title('Denoised & Reconstructed')
axes[2].axis('off')

plt.suptitle('Step 6: Nonlinear Filtering Result', fontsize=14)
plt.tight_layout()
plt.show()

## Step 7: Gamma Correction

Gamma correction adjusts overall brightness. A gamma value **< 1** brightens the image (useful for medical images that tend to be dark), while **> 1** darkens it.

In [ ]:
# ============================================================
# Step 7: Gamma Correction
# ============================================================
gamma_image = ImageEnhancer.gamma_correction(denoised_image, gamma=GAMMA)

print(f"Step 7: Gamma Correction applied (gamma={GAMMA})")
print(f"   Output dtype: {gamma_image.dtype}")
print(f"   Min: {gamma_image.min()}, Max: {gamma_image.max()}, Mean: {gamma_image.mean():.2f}")

# Display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(denoised_image, cmap='gray')
axes[0].set_title('Before Gamma Correction')
axes[0].axis('off')

axes[1].imshow(gamma_image, cmap='gray')
axes[1].set_title(f'After Gamma Correction (gamma={GAMMA})')
axes[1].axis('off')

plt.suptitle('Step 7: Gamma Correction', fontsize=14)
plt.tight_layout()
plt.show()

# Histogram comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 3))
axes[0].hist(denoised_image.ravel(), bins=256, color='steelblue', alpha=0.7)
axes[0].set_title('Before Gamma')
axes[0].set_xlabel('Pixel Intensity')

axes[1].hist(gamma_image.ravel(), bins=256, color='coral', alpha=0.7)
axes[1].set_title('After Gamma')
axes[1].set_xlabel('Pixel Intensity')

plt.suptitle('Step 7: Histogram Before/After Gamma', fontsize=14)
plt.tight_layout()
plt.show()

## Step 8: CLAHE Enhancement

**Contrast Limited Adaptive Histogram Equalization** enhances local contrast while preventing over-amplification of noise through a clip limit parameter.

In [ ]:
# ============================================================
# Step 8: CLAHE Enhancement
# ============================================================
clahe_image = ImageEnhancer.apply_clahe(gamma_image, clip_limit=CLAHE_CLIP, tile_grid_size=CLAHE_TILE)

print(f"Step 8: CLAHE applied (clip_limit={CLAHE_CLIP}, tile={CLAHE_TILE})")
print(f"   Output dtype: {clahe_image.dtype}")
print(f"   Min: {clahe_image.min()}, Max: {clahe_image.max()}, Mean: {clahe_image.mean():.2f}")

# Display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(gamma_image, cmap='gray')
axes[0].set_title('Before CLAHE')
axes[0].axis('off')

axes[1].imshow(clahe_image, cmap='gray')
axes[1].set_title(f'After CLAHE (clip={CLAHE_CLIP}, tile={CLAHE_TILE})')
axes[1].axis('off')

plt.suptitle('Step 8: CLAHE Enhancement', fontsize=14)
plt.tight_layout()
plt.show()

# Histogram comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 3))
axes[0].hist(gamma_image.ravel(), bins=256, color='steelblue', alpha=0.7)
axes[0].set_title('Before CLAHE')
axes[0].set_xlabel('Pixel Intensity')

axes[1].hist(clahe_image.ravel(), bins=256, color='coral', alpha=0.7)
axes[1].set_title('After CLAHE')
axes[1].set_xlabel('Pixel Intensity')

plt.suptitle('Step 8: Histogram Before/After CLAHE', fontsize=14)
plt.tight_layout()
plt.show()

## Step 9: Image Quality Metrics

Three metrics evaluate the enhancement:
- **CII** (Contrast Improvement Index): ratio of processed contrast to reference contrast
- **Entropy**: information content of the image
- **EME** (Effective Measure of Enhancement): local contrast measure across image blocks

In [ ]:
# ============================================================
# Step 9: Image Quality Metrics
# ============================================================
# Use calibrated_image as the reference for CII
mask = np.ones_like(calibrated_image)

cii = ImageMetrics.calculate_cii(clahe_image, calibrated_image, mask)
entropy = ImageMetrics.calculate_entropy(clahe_image)
eme = ImageMetrics.calculate_eme(clahe_image, 4, 4)
total_score = cii + entropy + eme

# Also compute metrics for the calibrated reference for comparison
entropy_ref = ImageMetrics.calculate_entropy(calibrated_image)
eme_ref = ImageMetrics.calculate_eme(calibrated_image, 4, 4)

print(f"Step 9: Image Quality Metrics")
print(f"{'':>20} {'Reference':>12} {'Processed':>12}")
print(f"   {'CII':>17}: {'1.0000':>12} {cii:>12.4f}")
print(f"   {'Entropy':>17}: {entropy_ref:>12.4f} {entropy:>12.4f}")
print(f"   {'EME':>17}: {eme_ref:>12.4f} {eme:>12.4f}")
print(f"   {'Total Score':>17}: {'N/A':>12} {total_score:>12.4f}")

# Bar chart comparing metrics
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

metrics_names = ['CII', 'Entropy', 'EME']
ref_vals = [1.0, entropy_ref, eme_ref]
proc_vals = [cii, entropy, eme]

for i, (name, rv, pv) in enumerate(zip(metrics_names, ref_vals, proc_vals)):
    axes[i].bar(['Reference', 'Processed'], [rv, pv], color=['steelblue', 'coral'])
    axes[i].set_title(name)
    axes[i].set_ylabel('Value')
    for j, v in enumerate([rv, pv]):
        axes[i].text(j, v, f'{v:.2f}', ha='center', va='bottom', fontsize=10)

plt.suptitle('Step 9: Quality Metrics Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## Step 10: Normalize & Resize

Normalize the processed image to 16-bit range (0-65535) and resize to the target output width while maintaining aspect ratio. Uses GPU-accelerated zoom.

In [ ]:
# ============================================================
# Step 10: Normalize & Resize
# ============================================================
# Normalize to 16-bit
img_gpu = cp.asarray(clahe_image)
min_val = cp.min(img_gpu)
max_val = cp.max(img_gpu)
img_normalized = min_val + ((img_gpu - min_val) / (max_val - min_val) * 65535)

# Resize
final_image = ImageResizer.resize(img_normalized, OUTPUT_WIDTH).get()

print(f"Step 10: Normalize & Resize completed")
print(f"   Normalized range: [{float(min_val):.2f}, {float(max_val):.2f}] -> [0, 65535]")
print(f"   Original size : {clahe_image.shape[1]}x{clahe_image.shape[0]}")
print(f"   Resized to    : {final_image.shape[1]}x{final_image.shape[0]}")
print(f"   Output dtype  : {final_image.dtype}")
print(f"   Min: {final_image.min():.2f}, Max: {final_image.max():.2f}, Mean: {final_image.mean():.2f}")

# Display
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(clahe_image, cmap='gray')
axes[0].set_title(f'Before Resize ({clahe_image.shape[1]}x{clahe_image.shape[0]})')
axes[0].axis('off')

axes[1].imshow(final_image, cmap='gray')
axes[1].set_title(f'Normalized & Resized ({final_image.shape[1]}x{final_image.shape[0]})')
axes[1].axis('off')

plt.suptitle('Step 10: Normalize & Resize', fontsize=14)
plt.tight_layout()
plt.show()

## Step 11: Final Comparison & Summary

Side-by-side comparison of every stage output and a complete summary of all parameters and results.

In [ ]:
# ============================================================
# Step 11: Final Comparison & Summary
# ============================================================

# --- Full pipeline visual summary (all 8 stages) ---
stages = {
    '1. Raw\nProjection': proj_image,
    '2. FFC': ffc_image,
    '3. Calibrated': calibrated_image,
    '4. Homomorphic\nFiltered': filtered_image,
    '5. Denoised\n(NLF)': denoised_image,
    '6. Gamma\nCorrected': gamma_image,
    '7. CLAHE\nEnhanced': clahe_image,
    '8. Final\n(Resized)': final_image,
}

fig, axes = plt.subplots(2, 4, figsize=(22, 11))
for ax, (title, img) in zip(axes.ravel(), stages.items()):
    ax.imshow(img, cmap='gray')
    ax.set_title(title, fontsize=12)
    ax.axis('off')

plt.suptitle('Full Pipeline - All Stages', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

# --- Summary table ---
print("=" * 65)
print("         FULL PIPELINE - PROCESSING SUMMARY")
print("=" * 65)
print(f"  Projection     : {PROJ_IMG_PATH}")
print(f"  Gain           : {GAIN_IMG_PATH}")
print(f"  Dark           : {DARK_IMG_PATH}")
print(f"  Calibration    : {CALIB_PATH}")
print(f"  Original size  : {proj_image.shape[1]}x{proj_image.shape[0]}")
print(f"  Calibrated size: {calibrated_image.shape[1]}x{calibrated_image.shape[0]}")
print(f"  Final size     : {final_image.shape[1]}x{final_image.shape[0]}")
print("-" * 65)
print(f"  FFC median     : {FFC_MEDIAN_SIZE}")
print(f"  BEMD           : max_iter={BEMD_MAX_ITER}, threshold={BEMD_THRESHOLD}, "
      f"window={BEMD_WINDOW_SIZE}, extrema={BEMD_EXTREMA_COUNT}")
print(f"  BIMFs extracted: {len(bimfs)}")
print(f"  Homomorphic    : D0={D0}, Rh={RH}, Rl={RL}")
print(f"  Nonlinear      : r={DENOISE_R}, beta={DENOISE_BETA}")
print(f"  Gamma          : {GAMMA}")
print(f"  CLAHE          : clip={CLAHE_CLIP}, tile={CLAHE_TILE}")
print("-" * 65)
print(f"  CII            : {cii:.4f}")
print(f"  Entropy        : {entropy:.4f}")
print(f"  EME            : {eme:.4f}")
print(f"  Total Score    : {total_score:.4f}")
print("=" * 65)

# --- Cleanup GPU memory ---
cp._default_memory_pool.free_all_blocks()
gc.collect()
print("\nGPU memory freed. Done!")